# TP1 - Grupo 14

André Lucena Ribas Ferreira - A94956

Paulo André Alegre Pinto - A97391

## Análise

Pretende-se alocar reuniões de projetos a dadas salas, dentro de determinados "timeslots".

Para tal decidimos usar uma família de variáveis binárias $x_{t,s,p,c}$ definidas com a seguinte semântica:

$$ x_{t,s,p,c} == 1 \quad \mbox{se e só se} \quad \mbox{o Colaborador $c$ estará Presente numa reunião do Projeto $p$ na Sala $s$ e no "Timeslot" $t$} $$

1. Para cada Timeslot, cada Líder tem de estar Presente nas reuniões do seu próprio Projeto.

Isto pode ser traduzido da seguinte maneira equivalente:

1. Para cada Timeslot, cada Colaborador está Presente na reunião de um Projeto numa Sala só se o seu Líder também estiver.

$$ tex $$

In [ ]:
for t in X:
    for s in X[t]:
        for p in X[t][s]:
            for c in X[t][s][p]:
                solver.Add(X[t][s][p][c] <= X[t][s][p][proj[p]['lider']])

2. Para cada Timeslot, cada Colaborador está Presente apenas numa reunião.
$$ tex $$

In [ ]:
for t in X:
    for c in range(C):
        solver.Add(sum([X[t][s][p][c] for s in X[t] for p in X[t][s]  if c in X[t][s][p]]) <= 1)

3. Para cada "Timeslot", cada Colaborador está Presente numa reunião só se tiver horário disponível.
$$ tex $$

De acordo com a definição que escolhemos para a nossa matriz de valores binários, não é possível um Colaborador $c$ participar de uma reunião na Sala $s$ do Projeto $p$ num "Timeslot" $t$ ao qual não tem disponibilidade. Isso deve-se tanto ao facto de sempre se iterar sobre os elementos do dicionário como de não existirem entradas $X[t][s][p][c]$ para Colaboradores sem disponibilidade.

4. Para cada "Timeslot", cada Sala apenas pode ter reuniões de um Projeto

Para verificar se um dado Projeto tem ou não reunião num dado "Timeslot", pode-se tirar partido da variável $x_{t,s,p,l}$, onde $l$ é o Líder do projeto $p$. Isto dá-se pelo facto que:

$$ x_{t,s,p,l} == 1 \quad \mbox{se e só se} \quad \mbox{o Projeto $p$ tem reunião no "Timeslot" $t$ na Sala $s$.} $$

$$ tex $$

In [ ]:
for t in X:
    for s in X[t]:
        solver.Add(sum([X[t][s][p][proj[p]['lider']] for p in X[t][s]]) <= 1)

In [4]:
from ortools.linear_solver import pywraplp

T, S, P, C = 6, 2, 5, 8    #período de 8 horas; 2 salas

projetos1 = {}
projetos1[0] = {'colaboradores':[0,1,2,4], 'lider':4, 'min':1}
projetos1[1] = {'colaboradores':[1,2,3], 'lider':1, 'min':2}
projetos1[2] = {'colaboradores':[0,2,3,4], 'lider':3, 'min':2}
projetos1[3] = {'colaboradores':[0,1,7], 'lider':7, 'min':3}
projetos1[4] = {'colaboradores':[2,3,4], 'lider':2, 'min':1}

matrix = {}
matrix[0] = [1,0,1,1,0,1]
matrix[1] = [1,1,1,1,0,1]
matrix[2] = [1,1,0,1,0,1]
matrix[3] = [1,1,1,0,1,1]
matrix[4] = [1,1,1,0,1,1]
matrix[5] = [1,0,1,0,1,1]
matrix[6] = [1,1,1,0,1,1]
matrix[7] = [1,1,1,0,0,1]

def scheduleStartUp(S, T, P, C, proj, disp):
    X = {}
    result = dict()
    solver = pywraplp.Solver.CreateSolver('SCIP')
    #dicionário com T*P*C*S variáveis -> X[t][p][c][s] == 1 sse c estará presente na reunião do projeto p no timeslot t na sala s.
    for t in range(T):
        X[t] = {}
        for s in range(S):
            X[t][s] = {}
            for p in range(P):
                if disp[proj[p]['lider']][t] == 1:
                    X[t][s][p] = {}
                    #Cada Colaborador apenas pode participar dos seus proj -> Obrigatório pela definição dos dicionários
                    for c in proj[p]['colaboradores']:
                        #Cada Colaborador tem de ter horário -> Obrigatório pela definição dos dicionários
                        if disp[c][t] == 1:
                            X[t][s][p][c] = solver.BoolVar('x[%i][%i][%i}[%i]' % (t,s,p,c))
                
    #Para cada Timeslot,
    for t in X:
        #Cada Sala apenas pode ter um projeto
        for s in X[t]:
            solver.Add(sum([X[t][s][p][proj[p]['lider']] for p in X[t][s]]) <= 1)
            
        #Cada Colaborador possível nesse timeslot pode ter, no máximo, 1 reunião
        for c in range(C):
            solver.Add(sum([X[t][s][p][c] for s in X[t] for p in X[t][s] if c in X[t][s][p]]) <= 1)
            
        #Cada Colaborador tem de ter horário -> Obrigatório pela definição dos dicionários
        #Cada Colaborador apenas pode participar dos seus proj -> Obrigatório pela definição dos dicionários
        
        #Cada Líder tem de participar do seu projeto OU SEJA
            #Cada Colaborador participa apenas de um projeto se o seu líder participar
        for s in X[t]:
            for p in X[t][s]:
                for c in X[t][s][p]:
                    solver.Add(X[t][s][p][c] <= X[t][s][p][proj[p]['lider']])
                    
        #Cada Projeto apenas pode ser selecionado se houver 'quórum', o projeto só existe se o líder existe
            #Apenas se pode olhar para uma dada sala, ou podemos ver salas de outros proj para algum colaborador
            #Número de participantes efetivos / total de colaboradores >= 1 / 2
                #Com cálculos chega-se a: 2*Número de participantes efetivos >= total de colaboradores
            #É necessário multiplicar pelo valor do líder para que não obriguemos a ter um somatório de 0's (pessoas não vão) >= quorum
        for s in X[t]:
            for p in X[t][s]:
                solver.Add(2*sum([X[t][s][p][c] for c in X[t][s][p]]) >= 
                           len(proj[p]['colaboradores']) * X[t][s][p][proj[p]['lider']])
        
    #Para cada Projeto, o número de vezes que foi selecionado tem de ser igual ou maior ao seu mínimo
    for p in range(P):
        solver.Add(sum([X[t][s][p][proj[p]['lider']] for t in X for s in X[t] if p in X[t][s]]) >= proj[p]['min'])
    
    #Número total de reuniões realizadas
    solver.Maximize(sum([X[t][s][p][proj[p]['lider']] for t in X for s in X[t] for p in X[t][s]]))
    
    status = solver.Solve()
    if status == pywraplp.Solver.OPTIMAL:
        #Limitar os mínimos aos máximos descobertos pela solução anterior
        for p in range(P):
            solver.Add(sum([X[t][s][p][proj[p]['lider']] for t in X for s in X[t] if p in X[t][s]]) >= 
                       sum([X[t][s][p][proj[p]['lider']].solution_value() for t in X for s in X[t] if p in X[t][s]]))
    
        #Média das reuniões por participante (total de participações / total de participantes, que é constante)
        solver.Minimize(sum([X[t][s][p][c] for t in X for s in X[t] for p in X[t][s] for c in X[t][s][p]]))
    
        status = solver.Solve()
    
    if status == pywraplp.Solver.OPTIMAL:
        count = 0
        for t in X:
            print(f'Slot: {t}')
            for s in X[t]:
                for p in X[t][s]:
                    if proj[p]['lider'] in X[t][s][p] and X[t][s][p][proj[p]['lider']].solution_value() == 1:
                        l = proj[p]['lider']
                        print(f'Projeto {p} de líder {l} na Sala {s}')
                        print("Colaboradores:")
                        for c in X[t][s][p]:
                            if X[t][s][p][c].solution_value() == 1:
                                print(f'{c}')
                                count += 1
                        print("")
            print("")
        print("Número médio de reuniões por colaborador: ")
        print(f"{count/C}")
    else:
        print("Nao é possivel construir o hórario")
        
scheduleStartUp(S, T, P, C, projetos1, matrix)

Slot: 0
Projeto 3 de líder 7 na Sala 0
Colaboradores:
1
7


Slot: 1
Projeto 3 de líder 7 na Sala 0
Colaboradores:
1
7

Projeto 4 de líder 2 na Sala 1
Colaboradores:
2
3


Slot: 2
Projeto 3 de líder 7 na Sala 0
Colaboradores:
0
7

Projeto 1 de líder 1 na Sala 1
Colaboradores:
1
3


Slot: 3
Projeto 1 de líder 1 na Sala 0
Colaboradores:
1
2


Slot: 4
Projeto 2 de líder 3 na Sala 0
Colaboradores:
3
4


Slot: 5
Projeto 2 de líder 3 na Sala 0
Colaboradores:
2
3

Projeto 0 de líder 4 na Sala 1
Colaboradores:
1
4


Número médio de reuniões por colaborador: 
2.25
